<a href="https://colab.research.google.com/github/vruddhis/nlp/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install indic-nlp-library
!pip install spacy
!python -m spacy download xx_sent_ud_sm



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 13.6 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 120.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('xx_sent_ud_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git


Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126 (from 1)
Receiving objects: 100% (139/139), 149.77 MiB | 14.18 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [3]:
from google.colab import files
uploaded = files.upload()


Saving marathi.txt to marathi.txt
Saving kannada.txt to kannada.txt
Saving hindi.txt to hindi.txt
Saving gujarati.txt to gujarati.txt


In [4]:
from indicnlp.tokenize import indic_tokenize
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from indicnlp import common
from indicnlp.tokenize import sentence_tokenize
import os

In [5]:
import spacy

In [6]:
INDIC_NLP_RESOURCES = "./indic_nlp_resources"
common.set_resources_path(INDIC_NLP_RESOURCES)

#this has hindi
nlp = spacy.load("xx_sent_ud_sm")

language_files = {
    "hi": "hindi.txt",
    "mr": "marathi.txt",
    "gu": "gujarati.txt",
    "kn": "kannada.txt"
}

custom_stopwords = {
    "mr": ["आहे", "आणि", "मध्ये", "च्या", "हा", "ही"],
    "gu": ["છે", "અને", "માં", "કે", "ના"],
    "kn": ["ಇದು", "ಮತ್ತು", "ನಲ್ಲಿ", "ಒಂದು"]
} #spacy does not have stopwords for these languages

In [7]:
def preprocess_text(text, lang):
    factory = IndicNormalizerFactory()
    normalizer = factory.get_normalizer(lang)
    normalized_text = normalizer.normalize(text)

    #sentence and word tokenization
    sentences = sentence_tokenize.sentence_split(normalized_text, lang)
    tokens = []
    for s in sentences:
        tokens.extend(indic_tokenize.trivial_tokenize(s))

    #stopword removal
    if lang == "hi":
        stopwords = nlp.Defaults.stop_words
    else:
        stopwords = set(custom_stopwords.get(lang, []))

    filtered_tokens = [t for t in tokens if t not in stopwords and t.strip() != ""]
    return filtered_tokens

In [8]:
for lang, filename in language_files.items():
    with open(filename, "r", encoding="utf-8") as f:
        text = f.read()

    tokens = preprocess_text(text, lang)
    print(f"\n==== {lang.upper()} ({filename}) ====")
    print("Total tokens:", len(tokens))
    print("Sample tokens:", tokens[:20])


==== HI (hindi.txt) ====
Total tokens: 1251
Sample tokens: ['इसराइली', 'मीडिया', 'ने', 'ग़ज़ा', 'सीज़फ़ायर', 'पर', 'चल', 'रही', 'बातचीत', 'के', 'प्रति', 'अमेरिकी', 'राष्ट्रपति', 'डोनाल्ड', 'ट्रंप', 'के', "'", 'सकारात्मक', 'रुख़', "'"]

==== MR (marathi.txt) ====
Total tokens: 917
Sample tokens: ['गावातल्या', 'आपल्या', 'अंगणवाडीचं', 'दार', 'सकाळी', 'नऊ', 'वाजता', 'उघडून', 'अंगणवाडी', 'ताई', 'दिवस', 'सुरु', 'करतात', '.', 'प्रत्यक्ष', 'दार', 'उघडल्यानंतर', 'पुढचा', 'टप्पा', 'असतो']

==== GU (gujarati.txt) ====
Total tokens: 1112
Sample tokens: ['વેપારક્ષેત્રે', 'આગળ', 'પડતું', 'નામ', 'ગણાતા', 'ગુજરાતમાં', 'શાળાકીય', 'શિક્ષણક્ષેત્રે', "'", 'ચિંતા', 'જન્માવે', "'", 'એવા', 'સમાચાર', 'થોડા', 'દિવસ', 'પહેલાં', 'સામે', 'આવ્યા', 'હતા']

==== KN (kannada.txt) ====
Total tokens: 164
Sample tokens: ['ಧರ್ಮಸ್ಥಳ', 'ಪ್ರಕರಣ', 'ಸಂಬಂಧ', 'ಬುರುಡೆ', 'ಚಿನ್ನಯ್ಯನ', 'ವಿರುದ್ಧ', 'ತನಿಖೆ', 'ಹಾಗೂ', 'ವಿಚಾರಣೆಯನ್ನು', 'ಬಹುತೇಕ', 'ಮುಕ್ತಾಯಗೊಳಿಸಿರುವ', 'ಎಸ್', 'ಐಟಿ', 'ಚಾರ್ಜ್', 'ಶೀಟ್', 'ಸಿದ್ಧಪಡಿಸುತ್ತಿದೆ', '.', 'ಈ', 'ತಿಂಗಳ', 'ಕ

In [9]:
for lang, filename in language_files.items():
    with open(filename, "r", encoding="utf-8") as f:
        text = f.read()
    tokens = preprocess_text(text, lang)
    with open(f"processed_{lang}.txt", "w", encoding="utf-8") as f:
        f.write(" ".join(tokens))
